In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

from tensorflow.keras import Sequential,layers,losses,utils,Input
from tensorflow.keras.layers import Dense,LSTM,Dropout,concatenate,Flatten, Conv1D, MaxPooling1D,Activation,RepeatVector,TimeDistributed
import tensorflow as tf
# from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional,GRU,Lambda,Dot,Concatenate
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error,mean_absolute_percentage_error

import matplotlib.pyplot as plt
import matplotlib

from sklearn.decomposition import PCA

In [3]:
data=pd.read_csv('changping.csv')

In [ ]:
data

In [ ]:
data.columns

In [6]:
col=[ 'year', 'month', 'day', 'hour', 'PM2.5', 'PM10', 'SO2', 'NO2',
       'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN',  'WSPM','wd']
datacol=data[col]

In [ ]:
datacol.isnull().sum()

In [8]:
# 直接删除缺失值
datashan=datacol.dropna()

In [ ]:
datashan.isnull().sum()

In [ ]:
datashan.shape

In [ ]:
datashan.describe()

In [ ]:
# 拼接时间为一列，绘制一年时间降水分布图
# datag['时间(年月日时)']=datag['年(年)'].map(str)+''+datag['月(月)'].map(str)+''+datag['日(日)'].map(str)+''+datag['时(时)'].map(str)
datashan['datatime']=datashan['year'].map(str)+'-'+datashan['month'].map(str)+'-'+datashan['day'].map(str)+'-'+datashan['hour'].map(str)

In [ ]:
datashan.head(8)

In [ ]:
# datag=data.copy()
datashan['datatime']=pd.to_datetime(datashan['datatime'],format='%Y-%m-%d-%H',errors='coerce')
datashan.index=datashan['datatime']

In [ ]:
datashan.head(8)

In [ ]:
datashan.isnull().sum()

In [17]:
chongcol=[ 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO',
       'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
datag=datashan[chongcol]

In [18]:
datal=datashan[chongcol]

In [ ]:
datal

In [ ]:
datag

以下不运行

In [ ]:
# 归一化
scaler = MinMaxScaler(feature_range=(0, 1))  # minmaxscaler 方法，用来做归一化
sel_col = datag.columns
# print(sel_col)
for col in sel_col:
    datag[col] = scaler.fit_transform(datag[col].values.reshape(-1, 1))  # reshape成为一列
print("Load dataset LEN: ", datag.shape[0])
print(sel_col)


In [ ]:
# 归一化后的数据
datag

In [23]:
# datag.corr()

In [ ]:
X = []
Y = []
columns_name = list(datag.columns)
y_index = columns_name.index('PM2.5')
tmp = np.array(datag, dtype=np.float32)
print("The shape of tmp is:")
print(tmp.shape)
pca = PCA(n_components=0.97)# 保证降维后的数据保持90%的信息
tmp2 = pca.fit_transform(tmp[:,1:])
print("The shape of tmp2 is:")
print(tmp2.shape)  # PCA降维y_index要改，或者先提取出来
print("input dim is:")
print(tmp2.shape[1])

In [129]:
# datax_trans=tmp2
# datay_trans=tmp[:,0]

In [130]:
# datax_trans

In [131]:
# datax_trans.shape

In [132]:
# datay_trans

In [133]:
# datay_trans.shape

In [25]:
seqX=120
seqY=1
day_delay=1
gap=1
X1=[]
for i in range(tmp2.shape[0] - (seqX+seqY+day_delay)*gap):
        # seqX, seqX+day_delay, seqX+day_delay+seqY
        # format: i+x*gap
        X.append(tmp2[i:(i + seqX*gap):gap, :])  # tmp2是PCA降维后的特征，如果是tmp那就是原始特征
        X1.append(tmp2[(i+(seqX+day_delay)*gap): (i+(seqX+day_delay+seqY)*gap): gap,:])
        Y.append(tmp[(i+(seqX+day_delay)*gap): (i+(seqX+day_delay+seqY)*gap): gap, y_index])
X = np.array(X)
Y = np.array(Y)
X1=np.array(X1)

划分数据集

In [54]:
# x_train, x_test,  y_train, y_test = train_test_split(datax_trans, datay_trans, test_size = 0.2, random_state = 666,shuffle=False)

In [ ]:
X.shape

In [ ]:
Y.shape

In [138]:
shendu_trainx, shendu_trainy = X[:int(0.8 * 32559)], Y[:int(0.8 * 32559)]  
jiqi_trainx, jiqi_trainy = X1[:int(0.8 * 32559)], Y[:int(0.8 * 32559)]  

In [139]:
shendu_testx, shendu_testy = X[int(0.8 * 32559):], Y[int(0.8 * 32559):] 
jiqi_testx, jiqi_testy = X1[int(0.8 * 32559):], Y[int(0.8 * 32559):] 

In [ ]:
shendu_trainx.shape

In [ ]:
jiqi_trainx.shape

In [ ]:
shendu_testx.shape

In [ ]:
jiqi_testx.shape

In [ ]:
jiqi_trainy.shape

In [ ]:
jiqi_testy.shape

转换成机器学习可以用的数据

In [149]:
realjiqi_trainx=jiqi_trainx.reshape(26047,7)

In [ ]:
realjiqi_trainx

In [153]:
realjiqi_testx=jiqi_testx.reshape(6512,7)

In [ ]:
realjiqi_testx

In [ ]:
jiqi_testx

In [ ]:
realjiqi_testx.shape

保存数据

In [165]:
# 保存深度学习训练测试数据
# 将训练集存起来
nptrainx=np.reshape(shendu_trainx,(-1,7))
pdtrainx1=np.reshape(nptrainx,(-1,120,7))

In [ ]:
pdtrainx1.shape

In [168]:
pdshendutrainx=pd.DataFrame(nptrainx,columns=['t1','t2','t3','t4','t5','t6','t7'])
pdshendutrainx.to_csv('shendux_train.csv',index=False)

In [169]:
nptestx=np.reshape(shendu_testx,(-1,7))

In [170]:
pdshendux_test=pd.DataFrame(nptestx,columns=['t1','t2','t3','t4','t5','t6','t7'])
pdshendux_test.to_csv('shendux_test.csv',index=False)

In [172]:
pdshenduy_train=pd.DataFrame(shendu_trainy,columns=['y1'])
pdshenduy_train.to_csv('shenduy_train.csv',index=False)

In [173]:
pdshenduy_test=pd.DataFrame(shendu_testy,columns=['y1'])
pdshenduy_test.to_csv('shenduy_test.csv',index=False)

In [174]:
# 保存机器学习训练测试集
pdjiqitrainx=pd.DataFrame(realjiqi_trainx,columns=['t1','t2','t3','t4','t5','t6','t7'])
pdjiqitrainx.to_csv('jiqix_train.csv',index=False)

In [175]:
pdjiqitestx=pd.DataFrame(realjiqi_testx,columns=['t1','t2','t3','t4','t5','t6','t7'])
pdjiqitestx.to_csv('jiqix_test.csv',index=False)

In [177]:
pdjiqiy_train=pd.DataFrame(jiqi_trainy,columns=['y1'])
pdjiqiy_train.to_csv('jiqiy_train.csv',index=False)

In [178]:
pdjiqiy_test=pd.DataFrame(jiqi_testy,columns=['y1'])
pdjiqiy_test.to_csv('jiqiy_test.csv',index=False)

In [ ]:
jiqi_testy

In [ ]:
shendu_testy

读取深度学习数据

In [192]:
# 读取训练集
dushendux_train=pd.read_csv('shendux_train.csv')
dushendux_train1=np.array(dushendux_train)
dushendux_train2=np.reshape(dushendux_train1,(-1,120,7))

dushenduy_train=pd.read_csv('shenduy_train.csv')
dushenduy_train2=np.array(dushenduy_train)
# 读取测试集
dushendux_test=pd.read_csv('shendux_test.csv')
dushendux_test1=np.array(dushendux_test)
dushendux_test2=np.reshape(dushendux_test1,(-1,120,7))

dushenduy_test=pd.read_csv('shenduy_test.csv')
dushenduy_test2=np.array(dushenduy_test)

In [ ]:
dushendux_train2

In [ ]:
dushendux_train2.shape

In [ ]:
dushendux_test2.shape

读取数据机器学习

In [182]:
# 读取训练集
dujiqix_train=pd.read_csv('jiqix_train.csv')
dujiqiy_train=pd.read_csv('jiqiy_train.csv')
# 读取测试集
dujiqix_test=pd.read_csv('jiqix_test.csv')
dujiqiy_test=pd.read_csv('jiqiy_test.csv')

In [ ]:
dujiqix_train

In [ ]:
dujiqix_test

In [ ]:
dujiqiy_train

In [ ]:
jiqi_trainy

In [ ]:
dux_test

In [ ]:
duy_train

In [ ]:
duy_test

方法一：线性预测

In [202]:
from sklearn.metrics import mean_absolute_error

In [89]:
from sklearn.linear_model import LinearRegression,RidgeCV,SGDRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score   #均方误差

In [344]:
# # # # 1.线性回归----此处使用基模型，参数均使用默认的，用于后面调参对比
# Lr=LinearRegression(normalize=False)
# Lr.fit(dujiqix_train,dujiqiy_train)
# xy_predict=Lr.predict(dujiqix_test)
# xy_modify = xy_predict*(xy_predict>=0)
# r2=r2_score(dujiqiy_test,xy_modify)
# # Mae=mean_squared_error(y_test,xy_modify)
# r2

In [ ]:
 # 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)

In [345]:
# xpreds = np.array(xy_modify)  # 转换为numpy形式
# xlabels = np.array(dujiqiy_test)
# xpreds= xpreds.reshape((-1,1))
# xfuture_len= xpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(xfuture_len):
#     xlabels[:, k] = xlabels[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
# for k in range(xfuture_len):
#     xpreds[:, k] = xpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [346]:
# np.sqrt(mean_squared_error(xlabels,xpreds))

In [347]:
# np.sqrt(mean_absolute_error(xlabels,xpreds))

In [343]:
# r2_score(xlabels,xpreds)

决策树

In [341]:
# tree_model=DecisionTreeRegressor()
# tree_model.fit(dux_train,duy_train)
# treey_pre=tree_model.predict(dux_test)
# ty_modify = treey_pre*(treey_pre>=0)
# jr=r2_score(duy_test,ty_modify)
# # treemae2=mean_squared_error(y_test,ty_modify2)
# # print(jr2,treemae2)
# jr

In [342]:
#  # 数据逆归一化
# maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
# print(maxmin)

In [349]:
# preds = np.array(ty_modify)  # 转换为numpy形式
# labels1 = np.array(duy_test)
# preds= preds.reshape((-1,1))
# future_len= preds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(future_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
# for k in range(future_len):
#     preds[:, k] = preds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
preds

In [ ]:
labels1

In [101]:
treemae=mean_squared_error(labels1,preds)

In [ ]:
treemae

In [ ]:
np.sqrt(mean_squared_error(labels1,preds))

In [ ]:
r2_score(labels1,preds)

随机森林

In [105]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rd=RandomForestRegressor(max_depth=11,max_features=6,min_samples_split=35,n_estimators=96,min_samples_leaf=9)     #基模型
rd.fit(dujiqix_train,dujiqiy_train)
suiy_pres=rd.predict(dujiqix_test)
yrand_modify = suiy_pres*(suiy_pres>=0)
suijir=r2_score(dujiqiy_test,yrand_modify)
suijir

In [368]:
label = np.array(dujiqiy_test)
for k in range(1):
#     print(k)
    label[:, k] = label[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.array(dujiqiy_test)

In [369]:
spreds = np.array(yrand_modify)  # 转换为numpy形式
# labels1 = np.array(dujiqiy_test)
spreds= spreds.reshape((-1,1))
sfuture_len= spreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(sfuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(sfuture_len):
    spreds[:, k] = spreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
spreds

In [ ]:
np.sqrt(mean_squared_error(label,spreds))

In [ ]:
mean_absolute_error(label,spreds)

保存预测结果

In [208]:
# spreds1=spreds.reshape(-1,1)

In [209]:
spreds1=spreds.reshape(-1,1)
baocunspreds=pd.DataFrame(spreds1,columns=['y_pred'])
baocunspreds.to_csv('随机森林预测结果.csv',index=False)

In [ ]:
a=pd.read_csv('随机森林预测结果.csv')
a

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('随机森林模型',fontsize=25)
ax.plot(labels1, "r", label="real", alpha=0.7,ms=7)
ax.plot(a, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文随机森林拟合图.jpg")

Adaboost算法

In [372]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
# 也可以对选择的弱回归器进行参数选择，方式为base_estimator__若回归器参数名
Ada_model=AdaBoostRegressor(DecisionTreeRegressor(max_depth=11, min_samples_split=35, min_samples_leaf=35),
                            n_estimators=50,learning_rate=0.1,loss='linear')
Ada_model.fit(dujiqix_train,dujiqiy_train)
aday_pres=Ada_model.predict(dujiqix_test)
aday_modify = aday_pres*(aday_pres>=0)
adar=r2_score(dujiqiy_test,aday_modify)
adar

In [ ]:
# 也可以对选择的弱回归器进行参数选择，方式为base_estimator__若回归器参数名
Ada_model1=AdaBoostRegressor(DecisionTreeRegressor(max_depth=11, min_samples_split=10, min_samples_leaf=30),
                            n_estimators=50,learning_rate=0.1,loss='square')
Ada_model1.fit(dujiqix_train,dujiqiy_train)
aday_pres1=Ada_model.predict(dujiqix_test)
aday_modify1 = aday_pres1*(aday_pres1>=0)
adar1=r2_score(dujiqiy_test,aday_modify1)
adar1

In [376]:
apreds = np.array(aday_modify)  # 转换为numpy形式
# labels1 = np.array(dujiqiy_test)
apreds= apreds.reshape((-1,1))
afuture_len= apreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(afuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(afuture_len):
    apreds[:, k] = apreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,apreds))

In [ ]:
r2_score(label,apreds)

In [ ]:
mean_absolute_error(label,apreds)

保存预测结果

In [381]:
apreds1=apreds.reshape(-1,1)
baocunapreds=pd.DataFrame(apreds1,columns=['y_pred'])
baocunapreds.to_csv('AdaBooost预测结果.csv',index=False)

In [ ]:
b=pd.read_csv('AdaBooost预测结果.csv')
b

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('AdaBoost模型',fontsize=25)
ax.plot(labels1, "r", label="real", alpha=0.7,ms=7)
ax.plot(b, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文AdaBoost拟合图.jpg")

GBRT

In [384]:
from sklearn.ensemble import GradientBoostingRegressor

In [385]:
# Gbrt_model= GradientBoostingRegressor(n_estimators=50,max_depth=7,min_samples_split=10,learning_rate=0.1,min_samples_leaf=5)#这里使用50个决策树
# Gbrt_model.fit(dujiqix_train,dujiqiy_train)
# gbrty_pres=Gbrt_model.predict(dujiqix_test)
# gbrty_modify= gbrty_pres*(gbrty_pres>=0)
# gbrtr=r2_score(dujiqiy_test,gbrty_modify)
# gbrtr

In [ ]:
Gbrt_model= GradientBoostingRegressor()#这里使用50个决策树
Gbrt_model.fit(dujiqix_train,dujiqiy_train)
gbrty_pres=Gbrt_model.predict(dujiqix_test)
gbrty_modify = gbrty_pres*(gbrty_pres>=0)
gbrtr=r2_score(dujiqiy_test,gbrty_modify)
gbrtr

In [388]:
gpreds = np.array(gbrty_modify)  # 转换为numpy形式
# labels2 = np.array(dujiqiy_test)
gpreds= gpreds.reshape((-1,1))
gfuture_len= gpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(gfuture_len):
#     labels2[:, k] = labels2[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(gfuture_len):
    gpreds[:, k] = gpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
gpreds.shape[1] 

In [ ]:
np.sqrt(mean_squared_error(label,gpreds))

In [ ]:
mean_absolute_error(label,gpreds)

In [ ]:
np.sum(np.absolute(labels1 - gpreds)) / len(dujiqiy_test)

In [ ]:
label

保存预测结果

In [310]:
gpreds=gpreds.reshape(-1,1)
baocungpreds=pd.DataFrame(gpreds,columns=['y_pred'])
baocungpreds.to_csv('GBRT预测结果.csv',index=False)

In [ ]:
c=pd.read_csv('GBRT预测结果.csv')
c

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('AdaBoost模型',fontsize=25)
ax.plot(labels, "r", label="real", alpha=0.7,ms=7)
ax.plot(c, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文GBRT拟合图.jpg")

xgboost

In [496]:
from xgboost import XGBRegressor

In [ ]:
other_params = {'learning_rate': 0.1, 'n_estimators': 75, 'max_depth': 4, 'min_child_weight': 5, 
                     'colsample_bytree': 0.7, 'gamma': 0.01,}
xgb_model1=XGBRegressor(**other_params)
xgb_model1.fit(dujiqix_train,dujiqiy_train)
xgby1_pred=xgb_model1.predict(dujiqix_test)
xgby1_modify = xgby1_pred*(xgby1_pred>=0)
xgbr1=r2_score(dujiqiy_test,xgby1_modify)
xgbr1

In [ ]:
other_params1 = {'learning_rate': 0.1, 'n_estimators': 70, 'max_depth': 4, 'min_child_weight': 5, 
                     'colsample_bytree': 0.7, 'gamma': 0.01,
                 }
xgb_model=XGBRegressor(**other_params1)
xgb_model.fit(dujiqix_train,dujiqiy_train)
xgby_pred=xgb_model.predict(dujiqix_test)
xgby_modify = xgby_pred*(xgby_pred>=0)
xgbr=r2_score(dujiqiy_test,xgby_modify)
xgbr

In [424]:
xgbpreds = np.array(xgby1_modify)  # 转换为numpy形式
# labels3 = np.array(dujiqiy_test)
xgbpreds= xgbpreds.reshape((-1,1))
xfuture_len= xgbpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(xfuture_len):
#     labels3[:, k] = labels3[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for j in range(xfuture_len):
    xgbpreds[:, j] = xgbpreds[:, j] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.array(xgby1_modify).reshape(-1,1)

In [ ]:
xgbpreds

In [ ]:
label

In [ ]:
np.sqrt(mean_squared_error(label,xgbpreds))

In [ ]:
mean_absolute_error(label,xgbpreds)

保存预测结果

In [448]:
xgbpreds=xgbpreds.reshape(-1,1)
baocunxgbpreds=pd.DataFrame(xgbpreds,columns=['y_pred'])
baocunxgbpreds.to_csv('XGBoost预测结果.csv',index=False)

In [ ]:
d=pd.read_csv('XGBoost预测结果.csv')
d

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('XGBoost模型',fontsize=25)
ax.plot(label, "r", label="real", alpha=0.7,ms=7)
ax.plot(d, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文XGBoost拟合图.jpg")

lightGBM

In [451]:
from lightgbm import LGBMRegressor
import lightgbm as lgb

In [ ]:
tmodel_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=36,
                              learning_rate=0.01, n_estimators=430, max_depth=8, 
                              metric='rmse', min_child_samples=4,
                            )#reg_alpha=0.5,reg_lambda=0.5,feature_fraction=0.6,bagging_fraction=0.4,
tmodel_lgb.fit(dujiqix_train,dujiqiy_train)
lgby=tmodel_lgb.predict(dujiqix_test)
lgby_modify=lgby*(lgby>=0)
lgbr=r2_score(dujiqiy_test,lgby_modify)
lgbr

In [ ]:
tmodel_lgb1 = lgb.LGBMRegressor(objective='regression',num_leaves=30,
                              learning_rate=0.01, n_estimators=500, max_depth=8, 
                              metric='rmse', min_child_samples=5,
                            )#reg_alpha=0.5,reg_lambda=0.5,feature_fraction=0.6,bagging_fraction=0.4,
tmodel_lgb1.fit(dujiqix_train,dujiqiy_train)
lgby1=tmodel_lgb1.predict(dujiqix_test)
lgby_modify1=lgby1*(lgby1>=0)
lgbr1=r2_score(dujiqiy_test,lgby_modify1)
lgbr1

In [489]:
lpreds1 = np.array(lgby_modify1)  # 转换为numpy形式
# llabel = np.array(dujiqiy_test)
lpreds1= lpreds1.reshape((-1,1))
lfuture_len= lpreds1.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(lfuture_len):
#     llabel[:, k] = llabel[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(lfuture_len):
    lpreds1[:, k] = lpreds1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,lpreds1))

In [ ]:
mean_absolute_error(label,lpreds1)

In [ ]:
mean_squared_error(llabel,lpreds)

保存预测结果

In [493]:
lpreds1=lpreds1.reshape(-1,1)
baocunlpreds=pd.DataFrame(lpreds1,columns=['y_pred'])
baocunlpreds.to_csv('LightGBM预测结果.csv',index=False)

In [ ]:
e=pd.read_csv('LightGBM预测结果.csv')
e

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('XGBoost模型',fontsize=25)
ax.plot(label, "r", label="real", alpha=0.7,ms=7)
ax.plot(e, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文LightGBM拟合图.jpg")

SVR

In [468]:
from sklearn.svm import SVR

In [ ]:
svr_rbf = SVR(kernel='linear', C=10, gamma=1)
svr_rbf.fit(dujiqix_train,dujiqiy_train)
svry_pred= svr_rbf.predict(dujiqix_test)
svry_modify= svry_pred*(svry_pred>=0)
svrr=r2_score(dujiqiy_test,svry_modify)
svrr

In [ ]:
svr_rbf2 = SVR(kernel='rbf', C=10, gamma=1)
svr_rbf2.fit(dujiqix_train,dujiqiy_train)
svry_pred2= svr_rbf2.predict(dujiqix_test)
svry_modify2 = svry_pred2*(svry_pred2>=0)
svrr2=r2_score(dujiqiy_test,svry_modify2)
svrr2

In [474]:
svrpreds = np.array(svry_modify2)  # 转换为numpy形式
# labels1 = np.array(y_test)
svrpreds= svrpreds.reshape((-1,1))
svrfuture_len= svrpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(sfuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(svrfuture_len):
    svrpreds[:, k] = svrpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,svrpreds))

In [ ]:
mean_absolute_error(label,svrpreds)

保存预测结果

In [477]:
svrpreds=svrpreds.reshape(-1,1)
baocunsvrpreds=pd.DataFrame(svrpreds,columns=['y_pred'])
baocunsvrpreds.to_csv('SVR预测结果.csv',index=False)

In [ ]:
f=pd.read_csv('SVR预测结果.csv')
f

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('SVR模型',fontsize=25)
ax.plot(label, "r", label="real", alpha=0.7,ms=7)
ax.plot(f, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文SVR拟合图.jpg")

lasso回归

In [480]:
from sklearn.linear_model import LassoCV

In [ ]:
La=LassoCV()
La.fit(dujiqix_train,dujiqiy_train)
lay_predict=La.predict(dujiqix_test)
lay_modify = lay_predict*(lay_predict>=0)
rla=r2_score(dujiqiy_test,lay_modify)
rla

In [483]:
lapreds = np.array(lay_modify)  # 转换为numpy形式
# labels1 = np.array(y_test)
lapreds= lapreds.reshape((-1,1))
lafuture_len= lapreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(lafuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(lafuture_len):
    lapreds[:, k] = lapreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,lapreds))

In [ ]:
mean_absolute_error(label,lapreds)

保存预测结果

In [486]:
lapreds=lapreds.reshape(-1,1)
baocunlapreds=pd.DataFrame(lapreds,columns=['y_pred'])
baocunlapreds.to_csv('Lasso预测结果.csv',index=False)

In [ ]:
g=pd.read_csv('Lasso预测结果.csv')
g

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Lasso模型',fontsize=25)
ax.plot(label, "r", label="real", alpha=0.7,ms=7)
ax.plot(g, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文Lasso拟合图.jpg")

深度学习算法

LSTM

In [6]:
# 读取训练集
dushendux_train=pd.read_csv('shendux_train.csv')
dushendux_train1=np.array(dushendux_train)
dushendux_train2=np.reshape(dushendux_train1,(-1,120,7))

dushenduy_train=pd.read_csv('shenduy_train.csv')
dushenduy_train2=np.array(dushenduy_train)
# 读取测试集
dushendux_test=pd.read_csv('shendux_test.csv')
dushendux_test1=np.array(dushendux_test)
dushendux_test2=np.reshape(dushendux_test1,(-1,120,7))

dushenduy_test=pd.read_csv('shenduy_test.csv')
dushenduy_test2=np.array(dushenduy_test)

In [7]:
# # # 构造批数据
def create_batch_dataset(x,y,train=True,buffer_size=1000,batch_size=64):#buffer_size=1000表示可以打乱窗口里面的数据
    batch_data=tf.data.Dataset.from_tensor_slices((tf.constant(x),tf.constant(y)))#数据封装，tensor类型
    if train:
        return batch_data.cache().shuffle(buffer_size).batch(batch_size)
    else:
        return batch_data.batch(batch_size)

In [8]:
train_data_single=create_batch_dataset(dushendux_train2, dushenduy_train2,train=True)
val_data_single=create_batch_dataset(dushendux_test2,dushenduy_test2,train=False)

In [ ]:
train_data_single

In [ ]:
dushendux_train2.shape[-2:]

In [ ]:
dushendux_train2.shape

In [ ]:
#以下为验证部分，与实验无关

In [ ]:
import numpy as np
a=np.array([[[3,1],[2,1],[1,1]]])
pppp=int(a.shape[2])
pppp

In [8]:
score_first_partq = Dense(pppp, use_bias=False, name='attention_score_vec1')(a)

In [27]:
for i in tf.compat.v1.trainable_variables():
    print(i)

In [ ]:
score_first_partq

In [ ]:
a

In [ ]:
h_tt = Lambda(lambda x: x[:, -1, :], output_shape=(pppp,), name='last_hidden_statee')(a)
h_tt

In [ ]:
scoree = Dot(axes=[1,2], name='attention_scoree')([h_tt, score_first_partq])
scoree

In [12]:
# 添加注意力机制
def attention_bilstm(inputs):
    """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28, philipperemy.
        """
    hidden_states = inputs
    hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
    score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
    h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
    score = Dot(axes=[1, 2], name='attention_score')([h_t, score_first_part])
    attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
    context_vector = Dot(axes=[1, 1], name='context_vector')([hidden_states, attention_weights])
    pre_activation = Concatenate(name='attention_output')([context_vector, h_t])
    attention_vector = Dense(128, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
    return attention_vector

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=7
model_input=Input(shape=(time_steps,input_dim))
bi_gru1=Bidirectional(LSTM(128,return_sequences=True))(model_input)
bi_gru2=Dropout(0.2)(bi_gru1)
bi_gru3=Bidirectional(LSTM(64,return_sequences=True))(bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru=attention_bilstm(bi_gru3)
# a=Dropout(0.2)(attbigru)
zhong=Dense(1)(attbigru)
attbilstm_model=Model(model_input,zhong)
attbilstm_model.compile(optimizer='adam', loss='mae')# metrics=['accuracy']
attbilstm_history = attbilstm_model.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1)

In [ ]:
# 预测
attbilstm_pres=attbilstm_model.predict(dushendux_test2,verbose=1)

In [ ]:
attbilstm_pres.shape

In [ ]:
attbilstm_modify = attbilstm_pres*(attbilstm_pres>=0)
lstmr1=r2_score(dushenduy_test2,attbilstm_modify)
lstmr1

In [ ]:
attbilstm_model.save('lunwensaveattbiLSTM_model1')